# pandapipes net from .csv tables
### using _plural_ create functions (e.g. `create_junctions`) instead of _singular_ ones ( e.g. `create_junction`)

It has been demonstrated in previous tutorials how a pandapipes net can be created step-by-step by creating each component individually with its respective `create...` function.
In many cases however, pandapipes will be integrated into a tool chain. For larger studies, grid data will likely be downloaded from somewhere, exported from other tools or might be received from another party. One established format for these exchanges is the [.csv-file format](https://en.wikipedia.org/wiki/Comma-separated_values).

This tutorial guides through the creation of a pandapipes net while taking example .csv-files as input.
To make the process more convenient and efficient, _plural_ create functions (e.g. create_junctions) are used.

The .csv files specifying the network are loaded from the "files/example_gas_net_csvs" folder which is located in the same folder as this notebook. They determine the [components](https://pandapipes.readthedocs.io/en/latest/components.html) of the example network used to illustrate this tutorial(see picture below): the junctions (i.e. nodes), the pipes, the external grids and the sinks.

The example network covers an imaginary development area and is designed as shown in the picture:

<img src="pics/gas_example_Longniddry.jpg" width="30%">

#### Import the necessary packages

In [ ]:
import pandas as pd
import os
import pandapipes as ppi
from pandapipes.plotting import simple_plot

#### Create an empty pandapipes network

In [ ]:
fluid = 'hgas'  # other possible fluids: water, lgas, hydrogen
net = ppi.create_empty_network('gas test network', fluid=fluid) # create empty network

#### Extract the data from the .csv files as pandas DataFrames

The data are extracted as DataFrames using the pandas function `read_csv`. Let's start with the junctions' data.

In [ ]:
pre_path = r".\files\example_gas_net_csvs"
junctions_df = pd.read_csv(os.path.join(pre_path,'example_net-junctions.CSV'))

Let's take a look at what is in the `junctions_df` DataFrame. The `head()` (from the pandas package) method displays the first 5 rows of the DataFrame, avoiding a crammed output.

In [ ]:
junctions_df.head()

The data of the other types of elements is also extracted

In [ ]:
pipes_df = pd.read_csv(os.path.join(pre_path, 'example_net-pipes.CSV'))
sinks_df = pd.read_csv(os.path.join(pre_path, 'example_net-sinks.CSV'))
ext_grids_df = pd.read_csv(os.path.join(pre_path, 'example_net-ext_grid.CSV'))
valves_df = pd.read_csv(os.path.join(pre_path, 'example_net-valves.CSV'))

pipes_df.head()

#### Create the network junctions and pipes based on the DataFrames
After having created an empty network, the next steps is always to create junctions. This is because all other elements are attached to junctions and cannot be created if their junctions do not yet exist.

We can now create all junctions based on the junctions-DataFrame at once, without any for-loop. Since no temperature is given, 10°C is assumed. For the plural-create-function, a single value, list or pandas-Series can be passed as input arguments.

In [ ]:
amb_temp = 283.15
geodata = junctions_df[['lon', 'lat']].values
junctions = ppi.create_junctions(net, nr_junctions=len(junctions_df), pn_bar=1, tfluid_k=amb_temp,
                         height_m=junctions_df['height'], geodata=geodata)

The .csv data do not specify any diameter for the pipes. Therefore, a uniform diameter of 0.05 m is assumed for all pipes. Passing a single value as an argument for the diameter assigns this diameter to all pipes. It is alternatively possible to pass an iterable to assign different diameters to different pipes - the iterable would need to have exactly the same length as the other passed iterables.
This is valid for all other relevant parameters for junctions, pipes, sinks, sources...

In [ ]:
pipes = ppi.create_pipes_from_parameters(net, pipes_df['from_junction'], pipes_df['to_junction'],
                                     length_km=pipes_df['length_km'], diameter_m=0.05, k_mm=0.2)

Note: There are two [functions to create pipes](https://pandapipes.readthedocs.io/en/latest/components/pipe/pipe_component.html) in pandapipes: `create_pipes_from_parameters` and `create_pipes` (and there are their equivalent _singular_ versions). The former creates pipes while passing the diameters as arguments (the user specifies the diameters while creating the pipes), while the latter creates pipes based on predefined [standard types](https://pandapipes.readthedocs.io/en/latest/standard_types/std_types_in_pandapipes.html).

Let's take a look at the pipes pandapipes table. The `head()` method can also be employed here. This table displays all the input parameters that specifies a pipe. Some are filled with default values since no specific data has been passed to the create function.

In [ ]:
net.pipe.head()

The input parameters can however be extended. Let's consider a slightly different .csv file for the pipes (namely _example_net-pipes_companies.CSV_) where the company owning each pipe is specified. This is not a parameter built in `create_pipes_from_parameters` function. This information can however be passed as keyword argument to the create function as follows:

In [ ]:
ppi.drop_pipes(net, net.pipe.index) # initially drop all the previously created pipes out of the network

pipes_df = pd.read_csv(os.path.join(pre_path,'example_net-pipes_companies.CSV')) # path to the csv file containing the pipes' and additionally the company owning/operating) them

ppi.create_pipes_from_parameters(net, pipes_df['from_junction'], pipes_df['to_junction'],
                                 length_km=pipes_df['length_km'], diameter_m=0.05, k_mm=0.2,
                                 company = pipes_df['company']) # "company" is declared as a new keyword argument
net.pipe.head()

The elements creation is done in the next code cell for the sinks, external grids and valves.

NB: The create functions return the indices of the elements they create. To avoid displaying this output below the cells, the output of these function is assigned to the variables `junctions`, `pipes`, `sinks`, `ext_grids`, and `valves`. These variables have therefore no functional purpose in the present tutorial, only a cosmetic one.

In [ ]:
sinks = ppi.create_sinks(net, sinks_df['junction'], mdot_kg_per_s=sinks_df['m_dot'])
ext_grids = ppi.create_ext_grids(net, junctions=ext_grids_df['junction'], p_bar=ext_grids_df['pressure'], t_k=amb_temp)
valves = ppi.create_valves(net, valves_df['from_junction'], valves_df['to_junction'], diameter_m=valves_df['diameter'])

Now, all the data from the csv-files has been converted into pandapipes-components.
It should be checked whether the network is actually feasible, i.e. a pipeflow can be properly calculated.

#### Run a pipeflow

The pipeflow function is responsible for computing the pressures at the nodes and flow rates through the pipes of the network.

In [ ]:
ppi.pipeflow(net)

#### Display the results of the pipeflow calculation

Running the pipeflow while among others create _res_ (for results) tables for every type of element. The results for the junctions  and the pipes are displayed as done in the next two cells.

In [ ]:
net.res_junction

In [ ]:
net.res_pipe.head()

By using pandas `concat` function, it is possible to display input parameters and results in the same DataFrame:

In [ ]:
pd.concat([net.pipe[["from_junction", "to_junction"]], net.res_pipe.v_mean_m_per_s], axis=1)

#### Plot the network
The parameters `plot_sinks` enable the plotting of sinks. The plotted size of the valves and junctions is tuned via the parameters `valve_size` and `junction_size`.

In [ ]:
simple_plot(net, plot_sinks=True, valve_size=0.6, junction_size=0.2)

#### Save and load the network

It is possible to save the network as .json file and to load it later. Loading a network from json is much faster than creating the whole network from csv-files again.

The network is saved as the file "from_csv_example_gas.json".

In [ ]:
ppi.to_json(net, "from_csv_example_gas.json")

In [ ]:
loaded_net = ppi.from_json("from_csv_example_gas.json")